<a href="https://colab.research.google.com/github/lephuocdat2000/DeepLearning-and-Application/blob/main/EngtoVie_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparing Data

###1, Import libraries

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers , activations , models , preprocessing , utils
import pandas as pd



###2, Reading the data

In [2]:
!wget http://www.manythings.org/anki/vie-eng.zip -O viet-eng.zip
!unzip viet-eng.zip

--2021-06-21 03:59:26--  http://www.manythings.org/anki/vie-eng.zip
Resolving www.manythings.org (www.manythings.org)... 172.67.173.198, 104.21.55.222, 2606:4700:3031::6815:37de, ...
Connecting to www.manythings.org (www.manythings.org)|172.67.173.198|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 298429 (291K) [application/zip]
Saving to: ‘viet-eng.zip’

viet-eng.zip        100%[===================>] 291.43K  1.78MB/s    in 0.2s    

2021-06-21 03:59:26 (1.78 MB/s) - ‘viet-eng.zip’ saved [298429/298429]

Archive:  viet-eng.zip
  inflating: _about.txt              
  inflating: vie.txt                 


In [3]:
lines = pd.read_table( 'vie.txt' , names=[ 'eng' , 'vie' ] )
lines.reset_index( level=0 , inplace=True)
lines.rename( columns={ 'index' : 'eng' , 'eng' : 'vie' , 'vie' : 'c' } , inplace=True )
lines = lines.drop('c',1)

In [ ]:
lines.head()

,eng,vie
0,Run!,Chạy!
1,Help!,Giúp tôi với!
2,Go on.,Tiếp tục đi.
3,Hello!,Chào bạn.
4,Hurry!,Nhanh lên nào!


### 3) Preparing input data for the Encoder ( `encoder_input_data` )

In [4]:
eng_lines = list()
for line in lines.eng:
    eng_lines.append( line ) 

tokenizer = preprocessing.text.Tokenizer()  
tokenizer.fit_on_texts( eng_lines ) 
tokenized_eng_lines = tokenizer.texts_to_sequences( eng_lines ) 

length_list = list()
for token_seq in tokenized_eng_lines:
    length_list.append( len( token_seq ))
max_input_length = np.array( length_list ).max()
padded_eng_lines = preprocessing.sequence.pad_sequences( tokenized_eng_lines , maxlen=max_input_length , padding='post' )
encoder_input_data = np.array( padded_eng_lines )

eng_word_dict = tokenizer.word_index
num_eng_tokens = len( eng_word_dict )+1

### 4) Preparing input data for the Decoder ( `decoder_input_data` )

In [13]:
!pip install underthesea

     |████████████████████████████████| 7.5MB 6.8MB/s 
     |████████████████████████████████| 1.3MB 49.7MB/s 
     |████████████████████████████████| 245kB 46.5MB/s 
     |████████████████████████████████| 747kB 47.1MB/s 
     |████████████████████████████████| 753.2MB 24kB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
     |████████████████████████████████| 901kB 41.0MB/s 
     |████████████████████████████████| 2.9MB 44.4MB/s 
     |████████████████████████████████| 1.1MB 40.8MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16184 sha256=72798eb134a0998fd7403ff0ce114a0b3b68f9c57a16d5ad6526758f0a1caa09
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval
ERROR: torchvision 0.10.0+cu102 has requirement torch==1.9.0, but you'll have torch 1.5.1 which is incompatible.
ERROR: torchtext 0.10.0 has requirement torch==1.9.0, but you'll have torch 1.5.1 which is incompat

In [15]:
from underthesea import word_tokenize
tokenized_vie_lines = []
dict_words = np.array([])
dict_numbers = np.array([]).astype('int64')
vie_lines = []
tokenized_vie_lines = []
vie_word_dict = dict()
for line in lines.vie:
    line = word_tokenize(line,format='text')  
    full_line = '<START> ' + line + ' <END>'
    words = full_line.split(" ")
    for word in words:
       where_word_arr = np.where(dict_words==word)[0]
       if len(where_word_arr)==0: 
           dict_words=np.append(dict_words,word)
           dict_numbers=np.append(dict_numbers,1)
       else: 
         dict_numbers[where_word_arr[0]]+=1
    vie_lines.append(full_line)

agr_sorted_dict_numbers = np.argsort(dict_numbers)[::-1]
for idx,value in enumerate(agr_sorted_dict_numbers):
    vie_word_dict.update({dict_words[value]:idx})

print(vie_word_dict)

# tokenizer = preprocessing.text.Tokenizer()
# tokenizer.fit_on_texts(vie_lines) 
# tokenized_vie_lines = tokenizer.texts_to_sequences(vie_lines) 

# length_list = list()
# for token_seq in tokenized_vie_lines:
#     length_list.append( len( token_seq ))

# max_output_length = np.array( length_list ).max()
# print( 'Vietnamese max length is {}'.format( max_output_length ))

# padded_vie_lines = preprocessing.sequence.pad_sequences( tokenized_vie_lines , maxlen=max_output_length, padding='post' )
# decoder_input_data = np.array( padded_vie_lines )
# print( 'Decoder input data shape -> {}'.format( decoder_input_data.shape ))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  del sys.path[0]


{'<START>': 0, '<END>': 1, '.': 2, 'Tôi': 3, 'không': 4, 'Tom': 5, 'là': 6, 'đã': 7, 'tôi': 8, '?': 9, 'có': 10, 'bạn': 11, 'ấy': 12, 'một': 13, 'của': 14, 'đó': 15, 'sẽ': 16, 'Bạn': 17, 'đi': 18, 'ở': 19, 'gì': 20, 'làm': 21, ',': 22, 'cho': 23, 'đang': 24, 'người': 25, 'nói': 26, 'được': 27, 'biết': 28, 'muốn': 29, 'phải': 30, 'này': 31, 'anh': 32, 'Anh': 33, 'đến': 34, 'với': 35, 'ta': 36, 'cái': 37, 'Mary': 38, 'có_thể': 39, 'nghĩ': 40, 'vào': 41, 'điều': 42, 'trong': 43, 'rất': 44, 'về': 45, 'ra': 46, 'để': 47, 'hơn': 48, 'nào': 49, 'bị': 50, 'Cô': 51, 'khi': 52, 'nhà': 53, 'đây': 54, 'nhiều': 55, 'rồi': 56, 'thích': 57, 'lại': 58, 'nó': 59, 'mà': 60, 'tiếng': 61, 'nên': 62, 'ai': 63, 'và': 64, 'ăn': 65, 'mình': 66, 'cô': 67, 'không_thể': 68, 'đâu': 69, 'thì': 70, 'những': 71, 'cần': 72, 'còn': 73, 'thấy': 74, 'Chúng_tôi': 75, 'cũng': 76, 'chưa': 77, 'chúng_tôi': 78, 'như': 79, 'học': 80, 'Có': 81, 'quá': 82, 'nữa': 83, 'từ': 84, 'tới': 85, 'cả': 86, 'vậy': 87, 'đều': 88, 'mua': 8

In [ ]:
for i,vie_line in enumerate(vie_lines):
    words = vie_line.split(" ")
    words_arr = np.zeros(len(words),dtype=int)
    for i,word in enumerate(words):
       words_arr[i] = vie_word_dict[word]
    tokenized_vie_lines.append(words_arr)

In [18]:
length_list = list()
for token_seq in tokenized_vie_lines:
    length_list.append( len( token_seq ))
max_output_length = np.array( length_list ).max()
padded_vie_lines = preprocessing.sequence.pad_sequences( tokenized_vie_lines , maxlen=max_output_length , padding='post' )
decoder_input_data = np.array( padded_vie_lines )
num_vie_tokens = len(dict_words )+1
print( 'Number of Vietnamese tokens = {}'.format( num_vie_tokens))

Number of Vietnamese tokens = 4107


### 5) Preparing target data for the Decoder ( decoder_target_data ) 

In [19]:
decoder_target_data = list()
for token_seq in tokenized_vie_lines:
    decoder_target_data.append( token_seq[ 1 : ] ) 
    
padded_vie_lines = preprocessing.sequence.pad_sequences( decoder_target_data , maxlen=max_output_length, padding='post' )
onehot_vie_lines = utils.to_categorical( padded_vie_lines , num_vie_tokens )
decoder_target_data = np.array( onehot_vie_lines )
print( 'Decoder target data shape -> {}'.format( decoder_target_data.shape ))

Decoder target data shape -> (7547, 38, 4107)


#Defining and Training the models

###1, Defining the Encoder - Decoder model

In [20]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( num_eng_tokens, 256 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 128 , return_state=True  )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( num_vie_tokens, 256 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 128 , return_state=True , return_sequences=True)
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( num_vie_tokens , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 256)    950272      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    1051392     input_2[0][0]                    
______________________________________________________________________________________________

###Training model

In [21]:
model.fit([encoder_input_data , decoder_input_data], decoder_target_data, batch_size=250, epochs=50 ) 
model.save( 'model.h5' ) 

Epoch 1/50
31/31 [==============================] - 19s 215ms/step - loss: 1.4918
Epoch 2/50
31/31 [==============================] - 7s 217ms/step - loss: 1.2272
Epoch 3/50
31/31 [==============================] - 7s 218ms/step - loss: 1.1727
Epoch 4/50
31/31 [==============================] - 7s 215ms/step - loss: 1.1252
Epoch 5/50
31/31 [==============================] - 7s 213ms/step - loss: 1.0852
Epoch 6/50
31/31 [==============================] - 7s 217ms/step - loss: 1.0518
Epoch 7/50
31/31 [==============================] - 7s 221ms/step - loss: 1.0239
Epoch 8/50
31/31 [==============================] - 7s 220ms/step - loss: 1.0004
Epoch 9/50
31/31 [==============================] - 7s 217ms/step - loss: 0.9794
Epoch 10/50
31/31 [==============================] - 7s 215ms/step - loss: 0.9587
Epoch 11/50
31/31 [==============================] - 7s 220ms/step - loss: 0.9379
Epoch 12/50
31/31 [==============================] - 7s 217ms/step - loss: 0.9172
Epoch 13/50
31/31 [=====

In [22]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 128 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 128 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( eng_word_dict[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=max_input_length , padding='post')

In [26]:

enc_model , dec_model = make_inference_models()

for epoch in range( encoder_input_data.shape[0] ):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter eng sentence : ' ) ) )
    #states_values = enc_model.predict( encoder_input_data[ epoch ] )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = vie_word_dict['<START>']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in vie_word_dict.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > max_output_length:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )


Enter eng sentence : Are you ok
 ốm ai cũng không ? <END> <START> ốm ! <END> <START> ốm <START> ốm ? <END> <START> ốm <START> ốm ? <END> <START> ốm <START> ốm ! <END> <START> ốm <START> ốm <START> ốm ! <END> <START> ốm <START>


KeyboardInterrupt: ignored

In [ ]:
np.where(dict_words=='<START>')[0]

array([0])